# Trying Out Numba

In [1]:
import numba
from numba import jit

In [2]:
%%time
n = 100000000

def add_a(n):
    a = 0
    for i in range(n):
        a += n


CPU times: user 4 µs, sys: 3 µs, total: 7 µs
Wall time: 12.4 µs


In [3]:
%%time
add_a(n)

CPU times: user 7.18 s, sys: 884 µs, total: 7.18 s
Wall time: 7.2 s


In [4]:
%%time
@jit(nopython=True)
def add_a_fast(n):
    a = 0
    for i in range(n):
        a += n

CPU times: user 212 µs, sys: 0 ns, total: 212 µs
Wall time: 219 µs


In [5]:
%%time
add_a_fast(n)

CPU times: user 1.01 s, sys: 1.81 s, total: 2.82 s
Wall time: 301 ms
